In [1]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps

In [2]:
data=pd.read_csv("play_tennis.csv")
print(data)

     Outlook Temperature Humidity    Wind Play Tennis
0      Sunny         Hot     High    Weak          No
1      Sunny         Hot     High  Strong          No
2   Overcast         Hot     High    Weak         Yes
3       Rain        Mild     High    Weak         Yes
4       Rain        Cool   Normal    Weak         Yes
5       Rain        Cool   Normal  Strong          No
6   Overcast        Cool   Normal  Strong         Yes
7      Sunny        Mild     High    Weak          No
8      Sunny        Cool   Normal    Weak         Yes
9       Rain        Mild   Normal    Weak         Yes
10     Sunny        Mild   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
13      Rain        Mild     High  Strong          No


In [3]:
# Entropy of the target attribute values
def find_entropy(df):
    target = df.keys()[-1]  # The last dataframe column is the target attribute (playGolf)
   
    entropy = 0
    values = df[target].unique()

    # for each value in the target playGolf attribute values
    for value in values:
        # ratio of values occurring and entropy
        fraction = df[target].value_counts()[value] / len(df[target])
        entropy += -fraction * np.log2(fraction)
    return entropy
entropy=print(find_entropy(data))

0.9402859586706311


## Attribute Entropy

In [4]:
# Entropy of attribute values
def find_entropy_attribute(df, attribute):
    target = df.keys()[-1]
    target_variables = df[target].unique()  # unique values in target playGolf attribute (Yes, No)
    variables = df[attribute].unique() # Identify Sunny, Overcast, Rainy

    # attribute entropy          # Variables=[sunny, sunny....5, overcast1.....overcast4, Rainy1...Ra5 ]
    entropy2 = 0
    # for each attribute value in attribute values
    for variable in variables:
        # value entropy
        entropy = 0
        # for each target value in target values (yes/no)
        for target_variable in target_variables:
            # frequency of attribute and target values (boolean indexing, pandas dataframe filtering)
            num = len(df[attribute][df[attribute] == variable][df[target] == target_variable])
            den = len(df[attribute][df[attribute] == variable])
            fraction = num / (den + eps)
            entropy += -fraction * np.log2(fraction + eps)
        fraction2 = den / len(df)
        entropy2 += -fraction2 * entropy
    return abs(entropy2)

    #return abs(entropy)

In [5]:
def bestClassifier(df):
    # Entropy_att = []
    # information gain array for all attributes
    IG = []
    # for all attributes excluding target
    for key in df.keys()[:-1]:
        # Entropy_att.append(find_entropy_attribute(df,key))
        # calculate and record information gain value
        IG.append(find_entropy(df) - find_entropy_attribute(df, key)) #0.940 -0.693= 0.247
    return df.keys()[:-1][np.argmax(IG)]  # IG[0.247, 0.029, 0.152, 0.048 ]


In [6]:
def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)


In [7]:
def ID3split(df, tree=None):
    target = df.keys()[-1]
      

    # Here we build our decision tree

    # Get attribute with maximum information gain
    node = bestClassifier(df) # 0.247

    # Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attributeValues = np.unique(df[node])

    # Create an empty dictionary to create tree (recursive-friendly definition)
    if tree is None:               #  Outlook ->root node attribute
        tree = {}
        tree[node] = {}

    # following loop recursively calls ID3split to create and add to the tree
    # it runs till the tree is pure (leaf (result) node branches are added to the tree)
    for value in attributeValues:
        # get the subtable from current node based on the value
        subtable = get_subtable(df, node, value)
        # get the most common target value in the subtable
        targetValues, counts = np.unique(subtable[target], return_counts=True)

        # if the subtable is empty, assign the leaf node to the most common target value
        if len(counts) == 1:
            tree[node][value] = targetValues[0]
        else:
            # recursively call ID3 to create subtrees
            tree[node][value] = ID3split(subtable)  # Calling the function recursively

    return tree

decisionTree = ID3split(data)
print(decisionTree)


{'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [8]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
data=pd.read_csv("play_tennis.csv")
print(data)
# Entropy of the target attribute values
def find_entropy(df):
    target = df.keys()[-1]  # The last dataframe column is the target attribute (playGolf)
   
    entropy = 0
    values = df[target].unique()

    # for each value in the target playGolf attribute values
    for value in values:
        # ratio of values occurring and entropy
        fraction = df[target].value_counts()[value] / len(df[target])
        entropy += -fraction * np.log2(fraction)
    return entropy
def bestClassifier(df):
    # Entropy_att = []
    # information gain array for all attributes
    IG = []
    # for all attributes excluding target
    for key in df.keys()[:-1]:
        # Entropy_att.append(find_entropy_attribute(df,key))
        # calculate and record information gain value
        IG.append(find_entropy(df) - find_entropy_attribute(df, key)) #0.940 -0.693= 0.247
    return df.keys()[:-1][np.argmax(IG)]  # IG[0.247, 0.029, 0.152, 0.048 ]
def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)
def ID3split(df, tree=None):
    target = df.keys()[-1]
      

    # Here we build our decision tree

    # Get attribute with maximum information gain
    node = bestClassifier(df) # 0.247

    # Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attributeValues = np.unique(df[node])

    # Create an empty dictionary to create tree (recursive-friendly definition)
    if tree is None:               #  Outlook ->root node attribute
        tree = {}
        tree[node] = {}

    # following loop recursively calls ID3split to create and add to the tree
    # it runs till the tree is pure (leaf (result) node branches are added to the tree)
    for value in attributeValues:
        # get the subtable from current node based on the value
        subtable = get_subtable(df, node, value)
        # get the most common target value in the subtable
        targetValues, counts = np.unique(subtable[target], return_counts=True)

        # if the subtable is empty, assign the leaf node to the most common target value
        if len(counts) == 1:
            tree[node][value] = targetValues[0]
        else:
            # recursively call ID3 to create subtrees
            tree[node][value] = ID3split(subtable)  # Calling the function recursively

    return tree
print("******************************************************************************")
decisionTree = ID3split(data)
print(decisionTree)


     Outlook Temperature Humidity    Wind Play Tennis
0      Sunny         Hot     High    Weak          No
1      Sunny         Hot     High  Strong          No
2   Overcast         Hot     High    Weak         Yes
3       Rain        Mild     High    Weak         Yes
4       Rain        Cool   Normal    Weak         Yes
5       Rain        Cool   Normal  Strong          No
6   Overcast        Cool   Normal  Strong         Yes
7      Sunny        Mild     High    Weak          No
8      Sunny        Cool   Normal    Weak         Yes
9       Rain        Mild   Normal    Weak         Yes
10     Sunny        Mild   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
13      Rain        Mild     High  Strong          No
******************************************************************************
{'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'Humidity': {'High